#  Image Classification using TensorFlow Convolutional Neural Network 

In [2]:
from __future__ import print_function
import numpy as np
import time
import math
import random
import os
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix
%matplotlib inline

## Imports 

In [3]:
import keras
from keras import layers 
from keras.models import Sequential 
from keras.layers import Conv2D
from keras.layers import MaxPooling2D
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers import Dense


## Load Data and transform them in a format compatible with keras library

In [4]:
from keras.datasets.cifar import load_batch
from keras import backend as K


def load_data():
    """Loads CIFAR10 dataset.
    # Returns
        Tuple of Numpy arrays: `(x_train, y_train), (x_test, y_test)`.
    """
    path = './cifar-10-batches-py'

    num_train_samples = 50000

    x_train = np.empty((num_train_samples, 3, 32, 32), dtype='uint8')
    y_train = np.empty((num_train_samples,), dtype='uint8')

    for i in range(1, 6):
        fpath = os.path.join(path, 'data_batch_' + str(i))
        (x_train[(i - 1) * 10000: i * 10000, :, :, :],
         y_train[(i - 1) * 10000: i * 10000]) = load_batch(fpath)

    fpath = os.path.join(path, 'test_batch')
    x_test, y_test = load_batch(fpath)

    y_train = np.reshape(y_train, (len(y_train), 1))
    y_test = np.reshape(y_test, (len(y_test), 1))

    if K.image_data_format() == 'channels_last':
        x_train = x_train.transpose(0, 2, 3, 1)
        x_test = x_test.transpose(0, 2, 3, 1)

    return (x_train, y_train), (x_test, y_test)

## Exploring the dataset - Define image size and number of classes

In [5]:
def params():
    pass 
    num_classes = 10       
    in_shape = (32, 32, 3) 
    batch_size = 32
    epochs = 10
    return [num_classes, in_shape, batch_size, epochs]

In [6]:
[num_classes, in_shape, batch_size, epochs] = params()

In [7]:
(x_train, y_train), (x_test, y_test) = load_data()
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'keras_cifar10_trained_model.h5'

# The data, split between train and test sets:
(x_train, y_train), (x_test, y_test) = load_data()

# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

## Building the CNN Model 

In [8]:
def build_model(in_shape):
    model = Sequential()
    model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=in_shape))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
    model.add(MaxPooling2D(pool_size=(2, 2)))
    model.add(Dropout(0.2))
    model.add(Flatten())
    model.add(Dense(256, activation='relu'))
    model.add(Dense(128, activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))
    return model

In [15]:
# Putting everything together.  
def train_cifar():
    optimizer = keras.optimizers.RMSprop(lr=1e-4)

    model = build_model(in_shape)
    model.compile(loss='categorical_crossentropy',
              optimizer=optimizer,
              metrics=['accuracy'])
    
    model.fit(x_train, y_train, 
            batch_size = batch_size,
            epochs = epochs,
            verbose = 1,
            validation_data=(x_test, y_test))
    
    return model
    
    
#    pass

model = train_cifar()

Train on 50000 samples, validate on 10000 samples
Epoch 1/10
50000/50000 [==============================] - 29s 581us/step - loss: 1.8829 - acc: 0.3002 - val_loss: 1.6638 - val_acc: 0.3882
Epoch 2/10
50000/50000 [==============================] - 38s 768us/step - loss: 1.5751 - acc: 0.4244 - val_loss: 1.4477 - val_acc: 0.4775
Epoch 3/10
50000/50000 [==============================] - 32s 635us/step - loss: 1.4336 - acc: 0.4782 - val_loss: 1.3465 - val_acc: 0.5143
Epoch 4/10
50000/50000 [==============================] - 35s 698us/step - loss: 1.3340 - acc: 0.5198 - val_loss: 1.2447 - val_acc: 0.5582
Epoch 5/10
50000/50000 [==============================] - 35s 710us/step - loss: 1.2556 - acc: 0.5505 - val_loss: 1.2016 - val_acc: 0.5702
Epoch 6/10
50000/50000 [==============================] - 33s 666us/step - loss: 1.1961 - acc: 0.5757 - val_loss: 1.1909 - val_acc: 0.5721
Epoch 7/10
50000/50000 [==============================] - 33s 662us/step - loss: 1.1430 - acc: 0.5947 - val_loss: 1.

In [18]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_7 (Conv2D)            (None, 30, 30, 32)        896       
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 15, 15, 32)        0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 13, 13, 64)        18496     
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 6, 6, 64)          0         
_________________________________________________________________
conv2d_9 (Conv2D)            (None, 4, 4, 128)         73856     
_________________________________________________________________
max_pooling2d_9 (MaxPooling2 (None, 2, 2, 128)         0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 2, 2, 128)         0         
__________

In [16]:
# Save model and weights
if not os.path.isdir(save_dir):
    os.makedirs(save_dir)
model_path = os.path.join(save_dir, model_name)
model.save(model_path)
print('Saved trained model at %s ' % model_path)

# Score trained model.
scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

Saved trained model at /Users/ftsitoura/UW_ML410/Lesson_4/saved_models/keras_cifar10_trained_model.h5 
10000/10000 [==============================] - 1s 137us/step
Test loss: 1.0450150201797486
Test accuracy: 0.6314
